## https://pbpython.com/pandas-grouper-agg.html

Pandas’ Grouper function and the updated agg function are really useful when aggregating and summarizing data.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('.\sample-salesv3.xlsx')
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
account number    1500 non-null int64
name              1500 non-null object
sku               1500 non-null object
quantity          1500 non-null int64
unit price        1500 non-null float64
ext price         1500 non-null float64
date              1500 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 82.2+ KB


In [4]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
dtype: object

## 월별로 ext price의 총 값을 보고싶다

resample함수를 사용하면 날자별 그룹핑을 편리하게 할수있다.
다만 resample함수를 사용하려면 df의 index가 date로 지정되어있어야한다.

In [5]:
df.set_index('date')

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26
...,...,...,...,...,...,...
2014-12-30 13:38:13,737550,"Fritsch, Russel and Anderson",S1-06532,12,55.80,669.60
2014-12-30 21:42:17,786968,"Frami, Hills and Schmidt",S1-06532,37,13.14,486.18
2014-12-30 22:45:19,239344,Stokes LLC,S2-10342,14,38.75,542.50


In [6]:
df.set_index('date').resample('M')['ext price'].sum()

date
2014-01-31    185361.66
2014-02-28    146211.62
2014-03-31    203921.38
2014-04-30    174574.11
2014-05-31    165418.55
2014-06-30    174089.33
2014-07-31    191662.11
2014-08-31    153778.59
2014-09-30    168443.17
2014-10-31    171495.32
2014-11-30    119961.22
2014-12-31    163867.26
Freq: M, Name: ext price, dtype: float64

## 이번에는 월별이 아니라 회사별로 ext price값의 합을 보고싶다!

In [8]:
df.groupby('name')['ext price'].sum()

name
Barton LLC                         109438.50
Cronin, Oberbrunner and Spencer     89734.55
Frami, Hills and Schmidt           103569.59
Fritsch, Russel and Anderson       112214.71
Halvorson, Crona and Champlin       70004.36
Herman LLC                          82865.00
Jerde-Hilpert                      112591.43
Kassulke, Ondricka and Metz         86451.07
Keeling LLC                        100934.30
Kiehn-Spinka                        99608.77
Koepp Ltd                          103660.54
Kuhn-Gusikowski                     91094.28
Kulas Inc                          137351.96
Pollich LLC                         87347.18
Purdy-Kunde                         77898.21
Sanford and Sons                    98822.98
Stokes LLC                          91535.92
Trantow-Barrows                    123381.38
White-Trantow                      135841.99
Will LLC                           104437.60
Name: ext price, dtype: float64

## 이번에는 회사별이면서 월별의 ext price값의 합을 보고싶다

In [9]:
df.set_index('date').groupby('name')['ext price'].resample('M').sum().head()

name        date      
Barton LLC  2014-01-31     6177.57
            2014-02-28    12218.03
            2014-03-31     3513.53
            2014-04-30    11474.20
            2014-05-31    10220.17
Name: ext price, dtype: float64

## 위 코드를 좀더 intuitive하고 간단하게 해결할수있는 방법은 grouper를 이용하는 것.

In [10]:
df.groupby(['name', pd.Grouper(key='date', freq='M')])['ext price'].sum().head()

name        date      
Barton LLC  2014-01-31     6177.57
            2014-02-28    12218.03
            2014-03-31     3513.53
            2014-04-30    11474.20
            2014-05-31    10220.17
Name: ext price, dtype: float64

## 이제부턴 agg함수의 활용

### let’s say we need to get the total of the ext price and quantity column as well as the average of the unit price . The process is not very convenient:

In [11]:
df[['ext price', 'quantity']].sum()

ext price    2018784.32
quantity       36463.00
dtype: float64

In [12]:
df['unit price'].mean()

55.007526666666664

## agg 함수를 이용하면 위 2줄의 코드를 한번에 끝낼수있다

In [13]:
df[['ext price', 'quantity', 'unit price']].agg(['sum', 'mean'])

,ext price,quantity,unit price
sum,2.018784e+06,36463.000000,82511.290000
mean,1.345856e+03,24.308667,55.007527


In [14]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean']})

,ext price,quantity,unit price
mean,1.345856e+03,24.308667,55.007527
sum,2.018784e+06,36463.000000,NaN


## sum, mean과 같이 python에서 제공하는 함수 말고 내가 만든 함수도 agg에 적용시킬 수 있다!

In [15]:
get_max = lambda x: x.value_counts(dropna=False).index[0]

In [16]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

,ext price,quantity,unit price,sku
<lambda>,NaN,NaN,NaN,S2-77896
mean,1.345856e+03,24.308667,55.007527,NaN
sum,2.018784e+06,36463.000000,NaN,NaN


In [17]:
get_max.__name__ = "most frequent sku"

In [18]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

,ext price,quantity,unit price,sku
mean,1.345856e+03,24.308667,55.007527,NaN
most frequent sku,NaN,NaN,NaN,S2-77896
sum,2.018784e+06,36463.000000,NaN,NaN


The aggregate function using a dictionary is useful but one challenge is that it does not preserve order. If you want to make sure your columns are in a specific order, you can use an OrderedDict :

In [19]:
import collections
f = collections.OrderedDict([('ext price', ['sum', 'mean']), ('quantity', ['sum', 'mean']), ('sku', [get_max])])
df.agg(f)

,ext price,quantity,sku
mean,1.345856e+03,24.308667,NaN
most frequent sku,NaN,NaN,S2-77896
sum,2.018784e+06,36463.000000,NaN
